In [19]:
import pandas as pd
f = open("c:/data/text/news1.txt", encoding='utf-8')
lines = f.readlines()
f.close()

In [20]:
import re
from konlpy.tag import Hannanum
han = Hannanum()
word_list = []
for i in range(len(lines)):
    sentence = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', lines[i])
    a = sentence.strip()
    n = han.nouns(a)
    n2 = [x for x in n if len(x) > 1]
    if len(n2) > 0:
        word_list.append(n2)

In [21]:
#역토큰화
detokenized_doc = []
for i in range(len(word_list)):
    t = ' '.join(word_list[i])
    detokenized_doc.append(t)
detokenized_doc

['학년도 수시 정시 통합 방안 수능 무력화해 대입제도 학생부종합전형 학종 꼼수다 학종 불신 현실 외면 혼란 김경범 서울대 교수 국가교육회의 공정사회를위한국민모 공정사회 오전 정부서울청사 기자회견 국가교육회의 위원 김경범 서울대 서어서문학 교수 사퇴 촉구 교수 학종 옹호 공정한 대입제도 개편 걸림돌 주장 학년 대입제도 논란 확산 전날 청주 오송 고교학점제 연구학교 교원연수 행사 교수 미래 대입 전형 주제 발표 불씨 교수 국가교육회의 고등교육전문위원회 위원 참여 교수 자료집 수시 정시 통합 수능 성적 학생부 면접 평가하자 제안 수능 과목 절대평가 연계 주장 학년 고교학점제 학년 대입 개편 한다 현행 대입 수시 정시 구분 수시 정시 원서 접수 수시 준비 학기 학교 수업 비판 제기 교수 주장 선발 통합 교실 정상화 학생 학부모 교사 부담 대학 관계자 교사 학부모 의견 내놨다 고교 교사 환영 대학 입학관계자 학부모 부정적 서울 일반고의 교사 대입 전형 교사 학생 학부모 부담 학기 수업 파행 이번 방안 변화 필요 반면 서울 사립대 입학처장 고교별 수준 제각각 상황 수능 절대평가 변별력 약화 우수 학생 선발 대학들 수능 비교 요소 추가 지적 자녀 김모 서울 송파구 주변 대입 준비 사람들 수시 정시 선택 집중 경우 통합 이것저것 걱정 실제 교수 제안 현실화하기 학년 대입개편 통합 문제 논의 무산 통합 수능 학생부 칸막이 죽음 트라이앵글 우려 때문 학생들 선호 대학 수능 비교 포기 환경 교육부 관계자 시기통합안 교수 의견 학년 대입 교육부 논의한 내용 설명 전민희',
 '우체국 전산 오류 등록금 연세대 합격 취소 억울 호소 학생 주목 누구 잘못이냐 온라인상 갑론을박',
 '페이스북 연세대학교 대나무숲 페이지 우체국 전산 오류 대학교 입학금 입금 입학 취소 연대 측은 입금 확인 제때 우리 과실 내용 억울함 호소 게재',
 '수험생 우체국 전산 오류 자료 연세대 제출 입학 관련 문제사항 우체국 연대 입학 취소 처분 통보 대학 시간 열정 소용 토로',
 '온라인상 이슈 연세대 측

In [22]:
#TF-IDF행렬
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(detokenized_doc)
X.shape

(8, 244)

In [23]:
#SVD
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=5)
svd_model.fit(X)
len(svd_model.components_)

5

In [24]:
#토픽수 x 단어수
import numpy as np
np.shape(svd_model.components_)

(5, 244)

In [25]:
#토픽의 핵심 키워드 추출
terms = vectorizer.get_feature_names_out() 
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print('Topic %d:' % (idx + 1),
              [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_, terms)

Topic 1: [('우체국', 0.27574), ('연세대', 0.24455), ('등록금', 0.21755), ('입학', 0.21163), ('전산', 0.20183)]
Topic 2: [('해당', 0.24731), ('납부', 0.18536), ('등록금', 0.17018), ('수험생', 0.15551), ('문자', 0.14785)]
Topic 3: [('학교', 0.34707), ('교수', 0.24003), ('의견', 0.20707), ('측이', 0.19373), ('생각', 0.19373)]
Topic 4: [('교수', 0.29166), ('수능', 0.22685), ('대입', 0.19444), ('수시', 0.19444), ('통합', 0.19444)]
Topic 5: [('기준', 0.30113), ('청와대', 0.30113), ('동의', 0.30113), ('여명', 0.30113), ('게시판', 0.30113)]
